In [1]:
import sympy as sp
from sympy import sin,cos
import numpy as np
from sympy import diff
from IPython.display import display

In [2]:
from commons import SurfaceCurve,Surface,MethodOfOrthonormalFrames,CovariantDeriv,TangentVectorDiffOp

In [4]:
u,v = sp.symbols("u,v")
du,dv=sp.symbols("du,dv")

<pre>First we shall start with the case of a surface p(u, v) in the space. Let a tangent vector
X(t) be given at each point on curve p(t) = p(u(t), v(t)) on this surface. (We call
this a vector field along the curve.) Since X(t) is a vector in the space, it has three
components and by differentiating every component for t, we define X`(t) (or dX/dt ).
X`(t) also has three components, so we can assume that it is also a vector in the
space, but it is not necessarily tangent to the surface p(u, v). We write X`(t) as a sum
of a tangent vector and normal vector of this surface</pre>

![](./images/im3.4.1.png)

In [10]:
class CovariantDeriv:
    def __init__(self,surface_curve:SurfaceCurve):
        self.surface_curve=surface_curve
        self.curve=self.surface_curve.curve_param
        self.u=self.surface_curve.u
        self.v=self.surface_curve.v
        #Any parameterization. Not necesarilly by distance.
        self.t=self.surface_curve.s
        self.surface=self.surface_curve.surface
        self.ort=MethodOfOrthonormalFrames(self.surface)
        self.W_u,self.W_v=self.ort.calc_mat_W()
        self.e1=sp.Array(self.ort.e1)
        self.e2=sp.Array(self.ort.e2)
        self.e3=sp.Array(self.ort.e3)
        self.e1,self.e2,self.e3=[(x/sp.sqrt(np.dot(x,x))).simplify() for x in [self.e1,self.e2,self.e3]]
        
    def covariant_deriv(self,e1=None,e2=None):
        #X = X[0]*surface.e1 + X[1]*surface.e2
        
        #Covariant deriv <=> the tangent vector
        t=self.t
        u_t,v_t=self.surface_curve.curve_param
        
        e1,e2,e3=self.e1,self.e2,self.e3
        e1,e2,e3=[x.subs(self.u,u_t).subs(self.v,v_t).simplify() for x in [e1,e2,e3]]
        
        xi1,xi2,xi3=self.calc_X_deriv_xi()
        
        return xi1*e1+xi2*e2
    
    def calc_X_deriv_xi(self):
        t=self.t
        xi1,xi2,xi3 = sp.symbols("xi1,xi2,xi3")
        u_t,v_t=self.surface_curve.curve_param
        
        curve=[f.subs(self.u,u_t).subs(self.v,v_t).simplify() for f in self.surface.p]
        
        X = sp.Array([diff(f,t) for f in curve])
        X_d = sp.Array([diff(f,t) for f in X])
        e1,e2,e3=self.e1,self.e2,self.e3
        e1,e2,e3=[x.subs(self.u,u_t).subs(self.v,v_t).simplify() for x in [e1,e2,e3]]
        
        sols=sp.solve([xi1*e1[i]+xi2*e2[i]+xi3*e3[i]-X_d[i] for i in range(3)],
                      (xi1,xi2,xi3))
        
        return sols[xi1],sols[xi2],sols[xi3]

    def tangent_vector(self):
        return self.covariant_deriv()
    
    def normal_vector(self):
        t=self.t
        u_t,v_t=self.surface_curve.curve_param
        
        e1,e2,e3=self.e1,self.e2,self.e3
        e1,e2,e3=[x.subs(self.u,u_t).subs(self.v,v_t).simplify() for x in [e1,e2,e3]]
        
        xi1,xi2,xi3=self.calc_X_deriv_xi()
        
        return xi3*e3
    
    def X_deriv(self):
        t=self.t
        u_t,v_t=self.surface_curve.curve_param
        curve=[f.subs(self.u,u_t).subs(self.v,v_t).simplify() for f in self.surface.p]
        return sp.Array([diff(diff(f,t)) for f in curve])
    
    def problem_3_4_2_eq(self):
        curve=self.curve
        t=self.t
        xi1,xi2=self.calc_xi_i()
        A=self.ort.calc_mat_A()
        B=A.inv()
        
        w12u,w21u=self.W_u[0,1],self.W_u[1,0]
        w12v,w21v=self.W_v[0,1],self.W_v[1,0]
        u_t,v_t=self.surface_curve.curve_param
        e1,e2=sp.Array(self.e1),sp.Array(self.e2)
        w12u,w21u,w12v,w21v,e1,e2,xi1,xi2=[x.subs(self.u,u_t).subs(self.v,v_t).simplify() for x in 
                                           [w12u,w21u,w12v,w21v,e1,e2,xi1,xi2]]
        dxi1dt=diff(xi1,t)
        dxi2dt=diff(xi2,t)
        dudt,dvdt=diff(u_t,t),diff(v_t,t)
        g=self.surface.find_christoffels_symbols()
        
        
        d_du,d_dv=sp.symbols("d_du,d_dv")
        return dxi1dt+d_du*(g["uuu"]*xi1*dudt+g["uvu"]*xi1*dvdt+g["vuu"]*xi2*dudt+g["vvu"]*xi2*dvdt)+\
               dxi2dt+d_dv*(g["uuv"]*xi1*dudt+g["uvv"]*xi1*dvdt+g["vuv"]*xi2*dudt+g["vvv"]*xi2*dvdt)
        
    

In [11]:
t=sp.symbols('t',positive=True)
surf=Surface(u,v,[
    u,
    v,
    u**2+v**2
])
cur=[
    t, #u(t)
    -sp.sqrt(t) # v(t)
]
sc=SurfaceCurve(u,v,t,surf,cur)
cov = CovariantDeriv(sc)

In [12]:
cov = CovariantDeriv(sc)

In [13]:
sols=cov.calc_X_deriv_xi()
display(*sols)

4*t/sqrt(4*t**2 + 1)

-48*t**(9/2)*sqrt(16*t**4 + 16*t**3 + 8*t**2 + 4*t + 1)/(256*t**8 + 256*t**7 + 192*t**6 + 128*t**5 + 48*t**4 + 16*t**3 + 4*t**2) - 8*t**(5/2)*sqrt(16*t**4 + 16*t**3 + 8*t**2 + 4*t + 1)/(256*t**8 + 256*t**7 + 192*t**6 + 128*t**5 + 48*t**4 + 16*t**3 + 4*t**2) + sqrt(t)*sqrt(16*t**4 + 16*t**3 + 8*t**2 + 4*t + 1)/(256*t**8 + 256*t**7 + 192*t**6 + 128*t**5 + 48*t**4 + 16*t**3 + 4*t**2)

4*t*sqrt(64*t**6 + 64*t**5 + 48*t**4 + 32*t**3 + 12*t**2 + 4*t + 1)/(8*t**3*sqrt(16*t**4 + 8*t**2 + 1) + 8*t**2*sqrt(16*t**4 + 8*t**2 + 1) + 2*t*sqrt(16*t**4 + 8*t**2 + 1)) + sqrt(64*t**6 + 64*t**5 + 48*t**4 + 32*t**3 + 12*t**2 + 4*t + 1)/(8*t**3*sqrt(16*t**4 + 8*t**2 + 1) + 8*t**2*sqrt(16*t**4 + 8*t**2 + 1) + 2*t*sqrt(16*t**4 + 8*t**2 + 1))

In [14]:
print("DX/dt:")
DXdt=cov.covariant_deriv()
display(DXdt)
print("A_X:")
A_X=cov.normal_vector()
display(A_X)

DX/dt:


[4*t**(3/2)*(-48*t**(9/2)*sqrt(16*t**4 + 16*t**3 + 8*t**2 + 4*t + 1)/(256*t**8 + 256*t**7 + 192*t**6 + 128*t**5 + 48*t**4 + 16*t**3 + 4*t**2) - 8*t**(5/2)*sqrt(16*t**4 + 16*t**3 + 8*t**2 + 4*t + 1)/(256*t**8 + 256*t**7 + 192*t**6 + 128*t**5 + 48*t**4 + 16*t**3 + 4*t**2) + sqrt(t)*sqrt(16*t**4 + 16*t**3 + 8*t**2 + 4*t + 1)/(256*t**8 + 256*t**7 + 192*t**6 + 128*t**5 + 48*t**4 + 16*t**3 + 4*t**2))/sqrt(16*t**3 + 4*t + (4*t**2 + 1)**2) + 4*t/(4*t**2 + 1), (4*t**2 + 1)*(-48*t**(9/2)*sqrt(16*t**4 + 16*t**3 + 8*t**2 + 4*t + 1)/(256*t**8 + 256*t**7 + 192*t**6 + 128*t**5 + 48*t**4 + 16*t**3 + 4*t**2) - 8*t**(5/2)*sqrt(16*t**4 + 16*t**3 + 8*t**2 + 4*t + 1)/(256*t**8 + 256*t**7 + 192*t**6 + 128*t**5 + 48*t**4 + 16*t**3 + 4*t**2) + sqrt(t)*sqrt(16*t**4 + 16*t**3 + 8*t**2 + 4*t + 1)/(256*t**8 + 256*t**7 + 192*t**6 + 128*t**5 + 48*t**4 + 16*t**3 + 4*t**2))/sqrt(16*t**3 + 4*t + (4*t**2 + 1)**2), -2*sqrt(t)*(-48*t**(9/2)*sqrt(16*t**4 + 16*t**3 + 8*t**2 + 4*t + 1)/(256*t**8 + 256*t**7 + 192*t**6 + 128*

A_X:


[-2*t*(4*t*sqrt(64*t**6 + 64*t**5 + 48*t**4 + 32*t**3 + 12*t**2 + 4*t + 1)/(8*t**3*sqrt(16*t**4 + 8*t**2 + 1) + 8*t**2*sqrt(16*t**4 + 8*t**2 + 1) + 2*t*sqrt(16*t**4 + 8*t**2 + 1)) + sqrt(64*t**6 + 64*t**5 + 48*t**4 + 32*t**3 + 12*t**2 + 4*t + 1)/(8*t**3*sqrt(16*t**4 + 8*t**2 + 1) + 8*t**2*sqrt(16*t**4 + 8*t**2 + 1) + 2*t*sqrt(16*t**4 + 8*t**2 + 1)))*sqrt(16*t**4 + 8*t**2 + 1)/sqrt(64*t**6 + 64*t**5 + 48*t**4 + 32*t**3 + 12*t**2 + 4*t + 1), 2*sqrt(t)*(4*t*sqrt(64*t**6 + 64*t**5 + 48*t**4 + 32*t**3 + 12*t**2 + 4*t + 1)/(8*t**3*sqrt(16*t**4 + 8*t**2 + 1) + 8*t**2*sqrt(16*t**4 + 8*t**2 + 1) + 2*t*sqrt(16*t**4 + 8*t**2 + 1)) + sqrt(64*t**6 + 64*t**5 + 48*t**4 + 32*t**3 + 12*t**2 + 4*t + 1)/(8*t**3*sqrt(16*t**4 + 8*t**2 + 1) + 8*t**2*sqrt(16*t**4 + 8*t**2 + 1) + 2*t*sqrt(16*t**4 + 8*t**2 + 1)))*sqrt(16*t**4 + 8*t**2 + 1)/sqrt(64*t**6 + 64*t**5 + 48*t**4 + 32*t**3 + 12*t**2 + 4*t + 1), (4*t*sqrt(64*t**6 + 64*t**5 + 48*t**4 + 32*t**3 + 12*t**2 + 4*t + 1)/(8*t**3*sqrt(16*t**4 + 8*t**2 + 1) + 8*

In [15]:
print("X'")
X_d=cov.X_deriv()
display(X_d)

X'


[0, 1/(4*t**(3/2)), 2]

In [16]:
dXdt=(DXdt+A_X).simplify()

In [17]:
for t_i in [0,5,-1,0.5,2,-2]:
    print("t =",t_i)
    [(print(k[0]),display(k[1])) for k in [("expected:",X_d.subs(t,t_i)),("actual:",dXdt.subs(t,t_i))]]
    print()

t = 0
expected:


[0, zoo, 2]

actual:


[0, zoo, nan]


t = 5
expected:


...

[0, sqrt(2)/16, 2]


t = -2
expected:


[0, sqrt(2)*I/16, 2]

actual:


[0, sqrt(2)*I/16, 2]

![](./images/im3.4.2.png)